# Myc/Max — Random Forest Reproducible Notebook

This notebook reproduces Random Forest (RF) results and figures for the Myc/Max TF–DNA binding project. It follows the exact CLI workflow used in your repo (`torch_prep_kfold.py` → `run_model.py`) and adds notebook-native plots for:
- Regression scatter vs. experimental ΔΔG_bind
- Binary confusion matrix (from a threshold on ΔΔG_bind)
- Feature importance (Gini and permutation)

**Before you run:**
1) Place your repository scripts at the project root (same directory structure as your docs):
```
project_root/
├─ Inputs/                 # you provide
│   ├─ REFERENCE.csv       # contains SEQUENCE_ID and LABEL_COL
│   └─ FEATURE*.csv        # 8 MMGBSA components (and corrections) averaged by frame
├─ Data/                   # will be created/used by scripts
├─ Model/                  # models saved here
├─ torch_prep_kfold.py
├─ run_model.py
└─ hyperopt.py             # optional
```
2) Activate the correct env (pandas, numpy, scikit-learn, scipy, matplotlib, seaborn optional).
3) Update the **CONFIG** cell below to your actual column names and filenames.


## CONFIG — set your paths and identifiers

In [20]:
# >>>>> EDIT THESE <<<<<
PROJECT_ROOT = "."

DATA_DIR = f"{PROJECT_ROOT}/ML_models/ML_RF/Data"
MODEL_DIR = f"{PROJECT_ROOT}/ML_models/ML_RF/Model"
INPUTS_DIR = f"{PROJECT_ROOT}/ML_models/ML_RF/Inputs"


# # --- base dirs ---
# REPO   = Path.cwd()
# MLRF   = REPO / "ML_models" / "ML_RF"       # we will run from here
# DATA   = MLRF / "Data"
# MODEL  = MLRF / "Model"
# INPUTS = MLRF / "Inputs"

# DATA.mkdir(parents=True, exist_ok=True)
# MODEL.mkdir(parents=True, exist_ok=True)

# # --- script path (yours is under Data) ---
# TORCH_PREP = MLRF / "Data" / "torch_prep_kfold.py"   # <- as you stated

###

# ID + label columns
SEQUENCE_ID = "sequence"      # e.g., 'sequence'
LABEL_COL   = "bind_avg"      # e.g., 'bind_avg' for regression

# Feature CSVs (provide one or more, joined on SEQUENCE_ID)
FEATURE_FILES = [
    f"{INPUTS_DIR}/rawdat.csv"
    # add more if you split per-term CSVs
]

REFERENCE_FILE = f"{INPUTS_DIR}/exp_data_all.csv"   # contains SEQUENCE_ID + LABEL_COL

# Split + CV knobs (align with best settings)
TEST_PERCENT = 0.20
KFOLD = 1
NUM_REPEATS = 1
KEEP_LAST_PERCENT = 0       # 0 = keep all
NAVG = 50                   # average rows per sequence in chunks
SCR_FRACTIONS = "0.0"       # naming uses only the first element

# RF hyperparameters (example starting point; update if you have best params)
N_ESTIMATORS = 300
MAX_DEPTH = None           # or an int (e.g., 10)
MAX_FEATURES = "auto"      # 'auto', 'sqrt', 0.5, etc.
MIN_SAMPLES_SPLIT = 2
MIN_SAMPLES_LEAF = 1
RANDOM_STATE = 42

# Prefix + model type
PREFIX = "mycmax"
MODEL_TYPE = "reg"          # 'reg', 'bin', or 'mclass'

# Plot/analysis knobs
BINARY_THRESHOLD = 0.0      # threshold on ΔΔG_bind for confusion matrix
DATA_SCALE = "log"          # if your run_model.py expects 'log' scaling flag


## 1) Initial split (merge features + hold-out test by sequence)

In [21]:
import os, subprocess, shlex, pathlib, sys

pathlib.Path(DATA_DIR).mkdir(exist_ok=True, parents=True)
pathlib.Path(MODEL_DIR).mkdir(exist_ok=True, parents=True)

# Build the CLI for initial split
cmd = f"""python {PROJECT_ROOT}/ML_models/ML_RF/Data/torch_prep_kfold.py --initial_split   --reference_file {REFERENCE_FILE}   --ref_id_col {SEQUENCE_ID}   --ref_label_col {LABEL_COL}   --filenames {' '.join(FEATURE_FILES)}   --feature_id_col {SEQUENCE_ID}   --prefix {PREFIX} --model_type {MODEL_TYPE}   --test_percentage {TEST_PERCENT}   --scramble_fractions {SCR_FRACTIONS}
"""
print(cmd)
# subprocess.run(shlex.split(cmd), check=False)  # uncomment to execute when your scripts are available

print("CWD:", os.getcwd())
res = subprocess.run(
    shlex.split(cmd),
    check=True,                # raise on failure
    capture_output=True,       # see stderr if it fails
    text=True
)
print(res.stdout)
print(res.stderr)


python ./ML_models/ML_RF/Data/torch_prep_kfold.py --initial_split   --reference_file ./ML_models/ML_RF/Inputs/exp_data_all.csv   --ref_id_col sequence   --ref_label_col bind_avg   --filenames ./ML_models/ML_RF/Inputs/rawdat.csv   --feature_id_col sequence   --prefix mycmax --model_type reg   --test_percentage 0.2   --scramble_fractions 0.0

CWD: /Users/nakku/Desktop/ML-Protein-DNA-Binding-Affinity/ML-Protein-DNA-Binding-Affinity

2025-09-08 16:16:32,435 - INFO - Loaded reference data: shape=(168, 2)
2025-09-08 16:16:32,609 - INFO - Combined feature data => shape=(272160, 10)
2025-09-08 16:16:32,620 - INFO - After merging => shape=(68040, 11)
2025-09-08 16:16:32,957 - INFO - Scr=0.0: train => mycmax_reg_scr0p00_trn_final.csv, shape=(53460, 10)
2025-09-08 16:16:32,958 - INFO - Scr=0.0: test  => mycmax_reg_scr0p00_tst_preprocess.csv, shape=(14580, 10)
2025-09-08 16:16:32,958 - INFO - Initial split completed. Exiting.



## 2) Build repeated K-folds for training (standardize using training stats)

In [ ]:
cmd = f"""python {PROJECT_ROOT}/ML_models/ML_RF/Data/torch_prep_kfold.py --process train   --keep_last_percent {KEEP_LAST_PERCENT}   --navg {NAVG}   --kfold {KFOLD} --num_repeats {NUM_REPEATS}   --prefix {PREFIX} --model_type {MODEL_TYPE}   --scramble_fractions {SCR_FRACTIONS}   --random_state {RANDOM_STATE}
"""
print(cmd)
# subprocess.run(shlex.split(cmd), check=False)



print("CWD:", os.getcwd())
res = subprocess.run(
    shlex.split(cmd),
    check=True,                # raise on failure
    capture_output=True,       # see stderr if it fails
    text=True
)
print(res.stdout)
print(res.stderr)

python ./ML_models/ML_RF/Data/torch_prep_kfold.py --process train   --keep_last_percent 0   --navg 50   --kfold 1 --num_repeats 1   --prefix mycmax --model_type reg   --scramble_fractions 0.0   --random_state 42

CWD: /Users/nakku/Desktop/ML-Protein-DNA-Binding-Affinity/ML-Protein-DNA-Binding-Affinity


CalledProcessError: Command '['python', './ML_models/ML_RF/Data/torch_prep_kfold.py', '--process', 'train', '--keep_last_percent', '0', '--navg', '50', '--kfold', '1', '--num_repeats', '1', '--prefix', 'mycmax', '--model_type', 'reg', '--scramble_fractions', '0.0', '--random_state', '42']' returned non-zero exit status 1.

In [ ]:
# --- args as a list ---
args = [
    sys.executable, str(TORCH_PREP),
    "--initial_split",
    "--reference_file", str(REFERENCE_FILE),
    "--ref_id_col", SEQUENCE_ID,
    "--ref_label_col", LABEL_COL,
    "--filenames", *[str(p) for p in FEATURE_FILES],
    "--feature_id_col", SEQUENCE_ID,
    "--prefix", PREFIX,
    "--model_type", MODEL_TYPE,
    "--test_percentage", str(TEST_PERCENT),
    "--scramble_fractions", SCR_FRAC,
]

print("Running from:", MLRF)
print("Script exists:", TORCH_PREP.exists())
res = subprocess.run(args, check=True, capture_output=True, text=True, cwd=str(MLRF))
print(res.stdout)
if res.stderr: print("[stderr]", res.stderr)

print("Expect outputs in:", (MLRF / "Data").resolve())

## 3) Train RF with cross‑validation (saves models + CV metrics)

In [8]:
cmd = f"""python {PROJECT_ROOT}/ML_models/ML_RF/run_model.py --mode 0   --model_type {MODEL_TYPE} --data_scale {DATA_SCALE}   --kfold {KFOLD} --num_repeats {NUM_REPEATS}   --model_dir {MODEL_DIR} --data_dir {DATA_DIR}   --output_file predictions   --ref_id_col {SEQUENCE_ID} --ref_label_col {LABEL_COL}   --n_estimators {N_ESTIMATORS} --max_depth {str(MAX_DEPTH)} --max_features {MAX_FEATURES}   --min_samples_split {MIN_SAMPLES_SPLIT} --min_samples_leaf {MIN_SAMPLES_LEAF}   --random_state {RANDOM_STATE}   --prefix {PREFIX} --scramble_fractions {SCR_FRACTIONS}
"""
print(cmd)
# subprocess.run(shlex.split(cmd), check=False)


python ./ML_models/ML_RF/run_model.py --mode 0   --model_type reg --data_scale log   --kfold 1 --num_repeats 1   --model_dir ./ML_models/ML_RF/Model --data_dir ./ML_models/ML_RF/Data   --output_file predictions   --ref_id_col sequence --ref_label_col bind_avg   --n_estimators 300 --max_depth None --max_features auto   --min_samples_split 2 --min_samples_leaf 1   --random_state 42   --prefix mycmax --scramble_fractions 0.0



<add some comments that allow people to see that they are infact running the 5 fold cross validation>

## 4) Prepare the test set using the training stats

In [9]:
cmd = f"""python {PROJECT_ROOT}/ML_models/ML_RF/Data/torch_prep_kfold.py --process test   --keep_last_percent {KEEP_LAST_PERCENT}   --navg {NAVG}   --prefix {PREFIX} --model_type {MODEL_TYPE}   --scramble_fractions {SCR_FRACTIONS}   --random_state {RANDOM_STATE}
"""
print(cmd)
# subprocess.run(shlex.split(cmd), check=False)


python ./ML_models/ML_RF/Data/torch_prep_kfold.py --process test   --keep_last_percent 0   --navg 50   --prefix mycmax --model_type reg   --scramble_fractions 0.0   --random_state 42



## 5) Evaluate saved models on the test set

In [10]:
cmd = f"""python {PROJECT_ROOT}/ML_models/ML_RF/run_model.py --mode 1   --model_type {MODEL_TYPE} --data_scale {DATA_SCALE}   --kfold {KFOLD} --num_repeats {NUM_REPEATS}   --model_dir {MODEL_DIR} --data_dir {DATA_DIR}   --output_file predictions   --ref_id_col {SEQUENCE_ID} --ref_label_col {LABEL_COL}   --random_state {RANDOM_STATE}   --prefix {PREFIX} --scramble_fractions {SCR_FRACTIONS}
"""
print(cmd)
# subprocess.run(shlex.split(cmd), check=False)


python ./ML_models/ML_RF/run_model.py --mode 1   --model_type reg --data_scale log   --kfold 1 --num_repeats 1   --model_dir ./ML_models/ML_RF/Model --data_dir ./ML_models/ML_RF/Data   --output_file predictions   --ref_id_col sequence --ref_label_col bind_avg   --random_state 42   --prefix mycmax --scramble_fractions 0.0



## 6) Notebook-native plots to match the paper/supplement

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path

# ---- Locate outputs produced by the pipeline ----
cv_pred_path = Path(DATA_DIR) / f"predictions_{MODEL_TYPE}_final_avg_scr0p00.csv"
tst_pred_path = Path(DATA_DIR) / f"predictions_test_{MODEL_TYPE}_scr0p00.csv"
cv_metrics_path = Path(PROJECT_ROOT) / f"final_metrics_{MODEL_TYPE}_trn_scr0p00.csv"
tst_metrics_path = Path(PROJECT_ROOT) / f"final_metrics_{MODEL_TYPE}_tst_scr0p00.csv"

print("CV predictions:", cv_pred_path.exists(), cv_pred_path)
print("Test predictions:", tst_pred_path.exists(), tst_pred_path)

# ---- Load predictions (sequence-level) ----
# Expected columns (example): sequence, y_true (experimental ΔΔG), y_pred (model ΔΔG)
def _load_preds(path):
    if path.exists():
        df = pd.read_csv(path)
        # Best guess for column names; adjust if your files use different names
        # Try to infer
        cols = df.columns.str.lower()
        if 'y_true' not in cols.tolist() or 'y_pred' not in cols.tolist():
            print("[Note] Please rename columns to include y_true and y_pred, or edit here.")
        return df
    else:
        print(f"[WARN] Missing: {path}")
        return None

cv_df = _load_preds(cv_pred_path)
tst_df = _load_preds(tst_pred_path)


CV predictions: False ML_models/ML_RF/Data/predictions_reg_final_avg_scr0p00.csv
Test predictions: False ML_models/ML_RF/Data/predictions_test_reg_scr0p00.csv
[WARN] Missing: ML_models/ML_RF/Data/predictions_reg_final_avg_scr0p00.csv
[WARN] Missing: ML_models/ML_RF/Data/predictions_test_reg_scr0p00.csv


### 6A) Regression scatter (ΔΔG_bind: predicted vs experimental)

In [ ]:
def plot_scatter(df, title="Regression (ΔΔG_bind)"):
    if df is None: 
        print("No dataframe provided.")
        return
    # Infer columns
    lc = df.columns.str.lower()
    y_true_col = df.columns[lc.get_loc('y_true')] if 'y_true' in lc else df.columns[-2]
    y_pred_col = df.columns[lc.get_loc('y_pred')] if 'y_pred' in lc else df.columns[-1]
    x = df[y_true_col].values
    y = df[y_pred_col].values

    pcc, _ = pearsonr(x, y)
    mse = mean_squared_error(x, y)
    plt.figure(figsize=(6,6))
    plt.scatter(x, y, alpha=0.7)
    plt.axhline(0, linestyle='--')
    plt.axvline(0, linestyle='--')
    plt.title(f"{title}\nPearson={pcc:.2f}, MSE={mse:.2f}")
    plt.xlabel("Experimental ΔΔG_bind")
    plt.ylabel("Predicted ΔΔG_bind")
    plt.show()

plot_scatter(cv_df, title="Training CV (averaged predictions)")
plot_scatter(tst_df, title="Held‑out Test")


### 6B) Binary confusion matrix (threshold on ΔΔG_bind)

In [ ]:
def plot_binary_cm(df, threshold=0.0, title="Binary classification (ΔΔG threshold)"):
    if df is None: 
        print("No dataframe provided.")
        return
    lc = df.columns.str.lower()
    y_true_col = df.columns[lc.get_loc('y_true')] if 'y_true' in lc else df.columns[-2]
    y_pred_col = df.columns[lc.get_loc('y_pred')] if 'y_pred' in lc else df.columns[-1]
    y_true = (df[y_true_col].values > threshold).astype(int)
    y_hat  = (df[y_pred_col].values > threshold).astype(int)
    cm = confusion_matrix(y_true, y_hat, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["non‑binding","binding"])
    disp.plot(values_format='d')
    plt.title(title)
    plt.show()

plot_binary_cm(cv_df, threshold=BINARY_THRESHOLD, title="Training CV (threshold=0.0)")
plot_binary_cm(tst_df, threshold=BINARY_THRESHOLD, title="Held‑out Test (threshold=0.0)")


### 6C) Feature importance (Gini + optional permutation)

In [ ]:
# If you saved per‑fold RF models, you can reload and average feature_importances_. 
# As a simple placeholder, this cell expects a CSV with per‑feature Gini importances if your pipeline produced it.
# Otherwise, adapt to load your .pkl models from Model/ and extract .feature_importances_.

import pandas as pd
from pathlib import Path

FEATURE_IMPORTANCE_CSV = Path(DATA_DIR) / "rf_feature_importances_scr0p00.csv"  # optional
if FEATURE_IMPORTANCE_CSV.exists():
    imp = pd.read_csv(FEATURE_IMPORTANCE_CSV)
    # Expect columns: feature, gini_importance
    imp = imp.sort_values('gini_importance', ascending=False)
    plt.figure(figsize=(7,4))
    plt.bar(imp['feature'], imp['gini_importance'])
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Gini importance")
    plt.title("Random Forest — Feature Importance (Gini)")
    plt.tight_layout()
    plt.show()
else:
    print(f"[Note] {FEATURE_IMPORTANCE_CSV} not found. If you have saved models, load them and compute importances here.")


## 7) Reproducibility report — capture config + env

In [ ]:
import sys, platform, json, pandas as pd
report = {
    "python": sys.version,
    "platform": platform.platform(),
    "config": {
        "PROJECT_ROOT": PROJECT_ROOT,
        "DATA_DIR": DATA_DIR,
        "MODEL_DIR": MODEL_DIR,
        "INPUTS_DIR": INPUTS_DIR,
        "SEQUENCE_ID": SEQUENCE_ID,
        "LABEL_COL": LABEL_COL,
        "TEST_PERCENT": TEST_PERCENT,
        "KFOLD": KFOLD,
        "NUM_REPEATS": NUM_REPEATS,
        "KEEP_LAST_PERCENT": KEEP_LAST_PERCENT,
        "NAVG": NAVG,
        "SCR_FRACTIONS": SCR_FRACTIONS,
        "RF": {
            "n_estimators": N_ESTIMATORS,
            "max_depth": MAX_DEPTH,
            "max_features": MAX_FEATURES,
            "min_samples_split": MIN_SAMPLES_SPLIT,
            "min_samples_leaf": MIN_SAMPLES_LEAF,
            "random_state": RANDOM_STATE
        },
        "MODEL_TYPE": MODEL_TYPE,
        "PREFIX": PREFIX,
        "DATA_SCALE": DATA_SCALE
    }
}
with open("/mnt/data/mycmax_rf_repro_config.json", "w") as f:
    json.dump(report, f, indent=2)
print("Saved config to /mnt/data/mycmax_rf_repro_config.json")